## Imports

In [85]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from google.colab import drive

## Load Data

In [78]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
rf_pred_data_path = '/content/drive/MyDrive/AIIM/Final/SSL data/pseudo_labeled_rf_full_real_conf_new.csv'
vime_pred_data_path = '/content/drive/MyDrive/AIIM/Final/SSL data/pseudo_labelled_data_vime.csv'
scarf_pred_data_path = '/content/drive/MyDrive/AIIM/Final/SSL data/pseudo_labelled_data_scarf.csv'
val_data_path = '/content/drive/MyDrive/AIIM/Final/SSL data/val_data.csv'

vime_val_data_path = '/content/drive/MyDrive/AIIM/Final/SSL data/pseudo_val_data_vime.csv'
scarf_val_data_path = '/content/drive/MyDrive/AIIM/Final/SSL data/pseudo_val_data_scarf.csv'

# read files
rf_data = pd.read_csv(rf_pred_data_path)
vime_data = pd.read_csv(vime_pred_data_path)
scarf_data = pd.read_csv(scarf_pred_data_path)
vime_val_data = pd.read_csv(vime_val_data_path)
scarf_val_data = pd.read_csv(scarf_val_data_path)
val_data = pd.read_csv(val_data_path)

# extract only 'Label' and 'Confidence' columns
y_full_rf = rf_data[['Label', 'Confidence']]
y_full_vime = vime_data[['Label', 'Confidence']]   # w/o val
y_full_scarf = scarf_data[['Label', 'Confidence']] # w/o val

# okay now put this aside. let's focus on val data now.

In [120]:
# sort 'val_data' by 'auto_id'
val_data = val_data.sort_values(by='auto_id')
# get ground true labels for val dataset
y_val = val_data['Label']

# get 'auto_id' for val_data
val_auto_id = val_data['auto_id']

# extract val data from RF datasets
rf_val_data = rf_data[rf_data['auto_id'].isin(val_auto_id)]

# Remove val data from rf_data
rf_data = rf_data[~rf_data['auto_id'].isin(val_auto_id)]
y_full_rf = rf_data[['Label', 'Confidence']]

# extract only 'Label' and 'Confidence'
y_val_rf = rf_val_data[['Label', 'Confidence']]
y_val_vime = vime_val_data[['Label', 'Confidence']]
y_val_scarf = scarf_val_data[['Label', 'Confidence']]

In [129]:
val_data

,ESR1,PGR,ERBB2,MKI67,PLAU,ELAVL1,EGFR,BTRC,FBXO6,SHMT2,...,Hormone Therapy,Neoplasm Histologic Grade,Cellularity,Surgery-breast conserving,Surgery-mastectomy,Label,DssTime,Event,auto_id,Confidence
5,7.536847,5.587666,11.514514,5.722951,6.741081,6.321480,5.466188,6.956486,7.673015,9.837096,...,0,2,1.0,0,1,1,36.266667,1,1002,1.0
56,10.395644,6.531288,9.075396,5.440774,7.861422,5.973844,5.757120,6.026611,7.666777,9.455256,...,1,2,0.5,1,0,0,86.066667,1,1003,1.0
90,6.204958,5.172111,8.881671,5.861609,8.530361,6.671294,11.724683,6.046692,7.401715,10.481299,...,0,3,0.0,0,1,1,8.066667,1,1004,1.0
72,10.384912,6.540470,10.855967,6.338578,8.053959,5.877256,5.440619,6.941926,7.574710,10.280946,...,1,2,1.0,0,1,1,51.766667,1,1013,1.0
71,7.253476,5.376423,11.856311,5.571443,8.600433,5.680836,6.572544,6.214799,7.747953,9.518145,...,0,3,0.0,0,1,1,17.933333,1,1018,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,11.563569,5.534383,9.898012,5.742905,8.866162,6.338569,6.053927,6.461317,7.517946,10.377976,...,1,3,0.5,1,0,1,42.966667,1,1451,1.0
45,6.074817,5.218277,12.545156,6.117034,10.579241,6.249009,6.086558,6.153012,8.119610,10.817736,...,1,3,0.5,0,1,1,49.533333,1,1454,1.0
4,11.424278,5.372667,10.864729,6.410957,6.787171,6.926255,5.685226,7.046461,6.946620,9.987552,...,1,2,0.5,1,0,1,28.866667,1,1455,1.0
65,8.690467,6.673764,9.979783,5.392179,8.183584,6.128115,6.666688,6.210970,7.272580,9.802586,...,1,2,0.5,0,1,0,126.666667,1,1459,1.0


In [121]:
#####------------ NOTE: Not necessary. Depends on want you want. -----------###

# Delete samples that have low conf in y_val_rf
# rows_with_confidence_05 = y_val_rf[y_val_rf['Confidence'] == 0.5].index.tolist()
# print(f"Number of deleted samples: {len(rows_with_confidence_05)}")
# y_val_rf = y_val_rf.drop(index=rows_with_confidence_05)
# y_val_rf = y_val_rf.reset_index(drop=True)
# y_val_vime = y_val_vime.drop(index=rows_with_confidence_05)
# y_val_vime = y_val_vime.reset_index(drop=True)
# y_val_scarf = y_val_scarf.drop(index=rows_with_confidence_05)
# y_val_scarf = y_val_scarf.reset_index(drop=True)

# rf_data = rf_data.drop(index=rows_with_confidence_05)
# rf_data = rf_data.reset_index(drop=True)

In [122]:
# Set the all confidence to that for class 0
y_val_rf.loc[y_val_rf['Label'] == 1, 'Confidence'] = 1 - y_val_rf['Confidence']
y_val_vime.loc[y_val_vime['Label'] == 1, 'Confidence'] = 1 - y_val_vime['Confidence']
y_val_scarf.loc[y_val_scarf['Label'] == 1, 'Confidence'] = 1 - y_val_scarf['Confidence']

# Do the same thing for the full data
y_full_rf.loc[y_full_rf['Label'] == 1, 'Confidence'] = 1 - y_full_rf['Confidence']
y_full_vime.loc[y_full_vime['Label'] == 1, 'Confidence'] = 1 - y_full_vime['Confidence']
y_full_scarf.loc[y_full_scarf['Label'] == 1, 'Confidence'] = 1 - y_full_scarf['Confidence']

In [54]:
y_val_vime

,Label,Confidence
0,0,0.533777
1,0,0.662941
2,0,0.636438
3,0,0.547237
4,1,0.494569
...,...,...
88,0,0.664638
89,1,0.342973
90,1,0.420558
91,0,0.634022


## Ensemble

### Involve all pseudo labels

In [99]:
import pandas as pd

def weighted_ensemble_prediction(data_rf, data_vime, data_scarf, weights):
    """
    Combine predictions and confidence scores from three models using weighted ensemble.

    Args:
        data_rf (pd.DataFrame): DataFrame containing predictions and confidence scores for the first model.
        data_vime (pd.DataFrame): DataFrame containing predictions and confidence scores for the second model.
        data_scarf (pd.DataFrame): DataFrame containing predictions and confidence scores for the third model.
        weights (list): List of weights for each model [weight_rf, weight_vime, weight_scarf].

    Returns:
        pd.DataFrame: DataFrame containing final predicted labels and weighted confidence scores.
    """
    # Ensure weights sum to 1 for proper scaling (optional)
    weights = [w / sum(weights) for w in weights]

    # Initialize an array to store final predictions
    final_predictions = []

    # Assume all dataframes are the same length
    n_samples = len(data_rf)

    for sample in range(n_samples):
        # Get the model outputs for the current sample (confidence of class 0)
        rf_prob = data_rf['Confidence'].iloc[sample]
        vime_prob = data_vime['Confidence'].iloc[sample]
        scarf_prob = data_scarf['Confidence'].iloc[sample]

        # Calculate the weighted probability for class 0
        weighted_prob = (
            rf_prob * weights[0] +
            vime_prob * weights[1] +
            scarf_prob * weights[2]
        )

        # Determine the final predicted class and confidence
        if weighted_prob >= 0.5:
            final_class = 0
            confidence = weighted_prob
        else:
            final_class = 1
            confidence = 1 - weighted_prob

        # Append the result
        final_predictions.append((final_class, confidence))

    # Create a DataFrame with the results
    result = pd.DataFrame(final_predictions, columns=['Label', 'Confidence'])
    return result


In [123]:
# RUN ON VAL DATASET FOR EVALUATION

# Define the weights for each model
weights = [0.2, 0.4, 0.4]

# Call the function with the data and weights
val_weighted_result = weighted_ensemble_prediction(y_val_rf, y_val_vime, y_val_scarf, weights)

# View the result
val_weighted_result

,Label,Confidence
0,1,0.529429
1,0,0.742244
2,1,0.691627
3,1,0.544649
4,1,0.543710
...,...,...
88,0,0.684768
89,1,0.621114
90,1,0.512507
91,0,0.602645


In [102]:
# EVALUATE

# compare 'weighted_result' to 'y_val'
accuracy = accuracy_score(y_val, val_weighted_result['Label'])
print(f"Accuracy: {accuracy}")

Accuracy: 0.7634408602150538


In [124]:
# Run on full data
weighted_result = weighted_ensemble_prediction(y_full_rf, y_full_vime, y_full_scarf, weights)
weighted_result

,Label,Confidence
0,0,1.000000
1,1,1.000000
2,0,1.000000
3,1,1.000000
4,0,1.000000
...,...,...
1535,0,0.767549
1536,0,0.663528
1537,0,0.599254
1538,0,0.690340


### Involve only high RF confidence

In [ ]:
# Weights for the models (try out different combination)
weights_three = [0.3, 0.4, 0.3]

n_classes = 2
n_samples = len(scarf_pred)

# Organize predictions into a list of samples, each containing model outputs
# samples = list(zip(rf_pred, vime_pred, scarf_pred))

# Initialize an array to store final predictions
final_predictions = []

for sample in range(n_samples):
    # Initialize a weighted probability for class 0
    weighted_probs = 0

    # Get the model outputs for the current sample (prob of class 0)
    rf_prob = rf_pred['Confidence'].iloc[sample]
    vime_prob = vime_pred['Confidence'].iloc[sample]
    scarf_prob = scarf_pred['Confidence'].iloc[sample]

    # Calculate the weighted probability for each class
    weighted_probs += rf_prob * weights_three[0]
    weighted_probs += vime_prob * weights_three[1]
    weighted_probs += scarf_prob * weights_three[2]

    # Determine the final predicted class based on maximum weighted probability
    if weighted_probs >= 0.5:
        final_class = 0
        final_predictions.append((final_class, weighted_probs))
    else:
        final_class = 1
        final_predictions.append((final_class, 1 - weighted_probs))

# Output the results
weighted_result = pd.DataFrame(final_predictions, columns=['Label', 'Confidence'])
weighted_result


,Label,Confidence
0,0,1.000000
1,1,1.000000
2,1,1.000000
3,0,1.000000
4,1,1.000000
...,...,...
1381,0,0.707003
1382,0,0.633023
1383,0,0.627764
1384,0,0.713382


In [ ]:
len(rows_with_confidence_05)

## Save the result

In [133]:
from typing_extensions import Final
# Concat everything
full_features = vime_data.drop(columns=['Label', 'Confidence'])
full_data = pd.concat([full_features, weighted_result], axis=1)

val_data['Confidence'] = 1.0
final_result = pd.concat([full_data, val_data], axis=0)
final_result = final_result.sort_values(by='auto_id')
final_result = final_result.reset_index(drop=True)
final_result = final_result.drop(columns=['auto_id'])

In [135]:
final_result

,ESR1,PGR,ERBB2,MKI67,PLAU,ELAVL1,EGFR,BTRC,FBXO6,SHMT2,...,Chemotherapy,Hormone Therapy,Neoplasm Histologic Grade,Cellularity,Surgery-breast conserving,Surgery-mastectomy,DssTime,Event,Label,Confidence
0,10.041281,7.376123,9.725825,5.427919,9.300307,6.219375,6.125355,5.888779,7.893369,9.007343,...,1,1,2.0,1.0,0.0,1.0,163.100000,1.0,0,1.000000
1,11.276581,7.331223,9.956267,5.629876,8.119906,5.665620,5.775809,6.251167,8.242063,10.871432,...,1,1,3.0,1.0,0.0,1.0,41.366667,1.0,1,1.000000
2,7.536847,5.587666,11.514514,5.722951,6.741081,6.321480,5.466188,6.956486,7.673015,9.837096,...,0,0,2.0,1.0,0.0,1.0,36.266667,1.0,1,1.000000
3,10.395644,6.531288,9.075396,5.440774,7.861422,5.973844,5.757120,6.026611,7.666777,9.455256,...,0,1,2.0,0.5,1.0,0.0,86.066667,1.0,0,1.000000
4,6.204958,5.172111,8.881671,5.861609,8.530361,6.671294,11.724683,6.046692,7.401715,10.481299,...,1,0,3.0,0.0,0.0,1.0,8.066667,1.0,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,11.628490,5.570690,10.475695,6.032211,9.944405,5.865408,5.703147,6.649948,7.272166,9.750208,...,0,1,3.0,1.0,0.0,1.0,NaN,NaN,0,0.767549
1629,10.879891,6.431113,10.219154,5.435795,9.224122,5.699195,5.825643,6.404899,7.385644,9.271953,...,0,1,3.0,1.0,0.0,1.0,NaN,NaN,0,0.663528
1630,9.591235,7.984515,9.935179,5.605596,9.799519,5.808704,5.905282,6.491419,7.865526,9.741103,...,0,1,3.0,1.0,1.0,0.0,NaN,NaN,0,0.599254
1631,11.055114,8.282737,9.892589,5.753274,8.687667,5.475813,5.587906,6.830579,8.468221,9.482622,...,0,1,2.0,0.5,0.0,1.0,NaN,NaN,0,0.690340


In [136]:
# Save to CSV
output_file = "/content/drive/My Drive/AIIM/Final/combined_pseudo_label_full_final.csv"
final_result.to_csv(output_file, index=False)
print(f"Combined labeled data saved to {output_file}")

Combined labeled data saved to /content/drive/My Drive/AIIM/Final/combined_pseudo_label_full_final.csv


In [ ]:
# # Weights for the models (try out different combination)
# weights_1 = [0.2, 0.4, 0.4]
# # weights_2 = [0.1, 0.5, 0.4]

# n_classes = 2
# n_samples = len(y_val_rf)

# # Initialize an array to store final predictions
# final_predictions = []

# for sample in range(n_samples):
#     # Initialize a weighted probability for class 0
#     weighted_probs = 0

#     # Get the model outputs for the current sample (prob of class 0)
#     rf_prob = y_val_rf['Confidence'].iloc[sample]
#     vime_prob = y_val_vime['Confidence'].iloc[sample]
#     scarf_prob = y_val_scarf['Confidence'].iloc[sample]

#     # Calculate the weighted probability for each class
#     weighted_probs += rf_prob * weights_1[0]
#     weighted_probs += vime_prob * weights_1[1]
#     weighted_probs += scarf_prob * weights_1[2]
#     # if rf_prob == 0.5:
#     #     weighted_probs += rf_prob * weights_2[0]
#     #     weighted_probs += vime_prob * weights_2[1]
#     #     weighted_probs += scarf_prob * weights_2[2]
#     # else:
#     #     weighted_probs += rf_prob * weights_1[0]
#     #     weighted_probs += vime_prob * weights_1[1]
#     #     weighted_probs += scarf_prob * weights_1[2]

#     # Determine the final predicted class based on maximum weighted probability
#     if weighted_probs >= 0.5:
#         final_class = 0
#         final_predictions.append((final_class, weighted_probs))
#     else:
#         final_class = 1
#         final_predictions.append((final_class, 1 - weighted_probs))

# # Output the results
# weighted_result = pd.DataFrame(final_predictions, columns=['Label', 'Confidence'])
# weighted_result
